In [1]:
import numpy as np
import scipy.integrate as integrate

In [2]:
# matrix contains all the evaluations of the functional appearing in the weak formulation with respect to term a(x)
# evaluated using Gauß-Lobatto-Integration
def MatrixAanbnq(p,q,adiscrete,BasisPhi,ArrayWeights,ArrayPoints,alpha):
    A = np.zeros((p-1,p-1))
    for i in range(p-1):
        for j in range(p-1):
                A[i][j] = -alpha * sum([ArrayWeights[k] * adiscrete[k] *\
                        np.polynomial.polynomial.polyval(ArrayPoints[k], np.polynomial.polynomial.polyder(BasisPhi[j].coef))*\
                        np.polynomial.polynomial.polyval(ArrayPoints[k], np.polynomial.polynomial.polyder(BasisPhi[i].coef)) \
                        for k in range(q)])
                A[i][j] = - alpha * A[i][j]
    return A

def calculateTheoreticalMatrixB(p,q,ArrayPoints,ArrayWeights, BasisPhi, bdiscrete):
    B = np.zeros([p-1,p-1])
    for i in range(p-1):
        for j in range(p-1):
            B[i,j] =  sum([ArrayWeights[k] * bdiscrete[k]* \
                    np.polynomial.polynomial.polyval(ArrayPoints[k], np.polynomial.polynomial.polyder(BasisPhi[j].coef)) * \
                    BasisPhi[i](ArrayPoints[k]) for k in range(q)])
    return B


def calculateTheoreticalMatrixC(p,q, ArrayPoints, ArrayWeights, BasisPhi, cdiscrete):
    C = np.zeros([p-1,p-1])
    for i in range(p-1):
        for j in range(p-1):
            C[i,j] = sum([ArrayWeights[k] * cdiscrete[k]* BasisPhi[i](ArrayPoints[k]) * \
                    BasisPhi[j](ArrayPoints[k]) for k in range(q)])
    return C


# transforms the Matrix A into a vector, row by row, assumes A is quadratic
def vectorize(A): 
    arr = np.array(A[0,:])
    for i in range(1,np.shape(A)[0]):
        arr = np.append(arr,A[i,:], axis = 0)
    return arr

# ATilde denotes a preconditioned variant of Matrix Aanbnq as proposed in the paper, improvement in 
# perfomrance basically non-existent
def MatrixATildeanbnq(p,q, adiscrete,BasisPhi,ArrayWeights, ArrayPoints,alpha):
    return np.matmul(np.linalg.inv(MatrixAanbnq(p, q, np.ones(q), BasisPhi, ArrayWeights, ArrayPoints, alpha)), 
                     MatrixAanbnq(p, q, adiscrete, BasisPhi, ArrayWeights, ArrayPoints, alpha))

# calculates a prior version of weight vector, according to paper
def cfnb(f,BasisPhi):
    cfnb = []
    for el in (BasisPhi):
        cfnb.append(integrate.quad(lambda x: f(x) * el(x),0,1)[0]) 
    return np.asarray(cfnb)

def cfnbTilde(f,BasisPhi,p,q,ArrayWeights,ArrayPoints,alpha):
    return np.matmul(np.linalg.inv(MatrixAanbnq(p, q, np.ones(len(ArrayPoints)), BasisPhi, ArrayWeights, ArrayPoints, -1)), \
                    cfnb(f,BasisPhi))

# calculates the weight vector used to calculate solution, u* = sum( cfnb * phi)        
def cunbnqa(f,BasisPhi,p,q,adiscrete,ArrayWeights,ArrayPoints,alpha):
    return np.matmul(np.linalg.inv(MatrixAanbnq(p, q, adiscrete, BasisPhi, ArrayWeights, ArrayPoints, -1)),cfnb(f,BasisPhi))    

def calculateTheoreticalSolutionDiffusionOnly(f,BasisPhi,p,q,adiscrete, ArrayWeights, ArrayPoints, alpha): 
    # returns the Array of weight coefficients
    BasisPolynom = [0]
    c = cunbnqa(f, BasisPhi, p, q, adiscrete, ArrayWeights, ArrayPoints, alpha)
    for i in range(p-1):
        BasisPolynom = np.polynomial.polynomial.polyadd(BasisPolynom, (c[i] * BasisPhi[i]).coef )
    return (BasisPolynom)

def calculateTheoreticalSolutionFlexible(cfnb, *args): 
# arguments are the matrices Aanbnq, B and C
    if(len(args) == 1):
        return np.matmul(np.linalg.inv(args[0]), cfnb) 
    else:
        return np.matmul(np.linalg.inv(sum(args)), cfnb)
    
# gets a weight vector and polynomial basis, returns the weighted sum (as polynomial)
def createPolynomial(BasisPhi,c):
    Basis = [0]
    for i in range(len(BasisPhi)):
        Basis =  np.polynomial.polynomial.polyadd(Basis , c[i] * BasisPhi[i].coef)
    return np.polynomial.polynomial.Polynomial(Basis)

# evaluates the polynomial pol at point x
def evaluateSolution(pol, x):
    return np.polynomial.polynomial.polyval(x, pol.coef)

